In [1]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import numpy as np
import time

Após pesquisar, descobri que webscrapping a partir de um usuários é ilegal (pelo o menos nos EUA), no entanto, se acessarmos os dados que estao publicamente disponíveis, não estaremos violando nenhuma lei.
Além disso, o link utilizado a baixo é para uma página do linkedin com filtros ja estabelecidos. São eles:
Vaga = analista de dados
Regiao = Sao Paulo, Sao Paulo, Brasil
Nivel de experiencia = Estagio, Assistente, Júnior
apenas do último mês

In [2]:
browser = webdriver.Chrome('chromedriver.exe')
browser.get('https://www.linkedin.com/jobs/search?keywords=Dados&location=Brasil&geoId=106057199&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0')

C:\Users\mateu\AppData\Local\Temp\ipykernel_3620\608424945.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('chromedriver.exe')


selecionar os links com os detalhes de cada emprego

In [3]:
element = browser.find_element(By.TAG_NAME, 'body')
last_height = browser.execute_script('return document.body.scrollHeight')
lista = []
links_endereco = []
quantidade_total = 1000 
while quantidade_total > len(lista):
    # vai ate o final da pagina
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    # scroll para cima para come;ar a carregar a prox pagina
    element.send_keys(Keys.PAGE_UP)
    # agurada carregar
    time.sleep(2)
    # scroll para baixo para simular uma pessoa
    element.send_keys(Keys.PAGE_DOWN)
    # procurar pelo botao "mais vagas".
    new_height = browser.execute_script('return document.body.scrollHeight')
    if last_height == new_height:
        mais_vagas = browser.find_element(By.CSS_SELECTOR, '#main-content > section.two-pane-serp-page__results-list > button')
        mais_vagas.click()
        time.sleep(2)
    last_height = new_height
    links_objeto = browser.find_elements(By.XPATH, '//a[contains(@href, "com/jobs/view")]') 
    for elemento in links_objeto:
        links_endereco.append(elemento.get_attribute('href'))
    lista = links_endereco


In [4]:
len(lista)

1091

Após analisar o site, identifiquei a classe que continha a informação necessária, agora para extrair utilizarei o selenium e suas funcionalidade de busca para extrair as informações.

In [9]:
#criação de listas para armazenar as informações
cargos = []
descricoes = []
categorias = []
empresas = []
links = []
locais = []
contador_01 = 0
contador_02 = 0

#loop para navegar, encontrar e extrair as informações desejadas.
for link in lista:
    try:
        #acessar o link
        browser.get(link)
        try:
            #procura no html as classes que contenham a identificação do elemento que procuro.
            cargo = browser.find_element(By.XPATH, '//h1[contains(@class, "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title")]') 
            #necessário extrair o texto, caso contrário teria um objeto
            cargos.append(cargo.text)
            #procura no html pelo botão
            exibir_mais = browser.find_element(By.CSS_SELECTOR, '#main-content > section.core-rail > div > div.decorated-job-posting__details > section.core-section-container.my-3.description > div > div.description__text.description__text--rich > section > button.show-more-less-html__button.show-more-less-html__button--more')
            #necessário clicar para exibir a informação completa
            exibir_mais.click() 
            #procura por uma classe em questão
            descricao = browser.find_element(By.XPATH, '//div[contains(@class, "show-more-less-html__markup")]') 
            #extrair o texto
            descricoes.append(descricao.text)
            #como existem vários elementos com esta mesma classe, é necessário utilizar o .finf_element*S*.
            categoria = browser.find_elements(By.XPATH, '//span[contains(@class, "description__job-criteria-text description__job-criteria-text--criteria")]') 
            #como a variável categoria é uma lista,é necessário utilizar um loop para extrair o texto de cada elemento.  
            lista_suporte = []
            for topico in categoria: 
                lista_suporte.append(topico.text)
            #cada página terá uma lista de elementos, na lista categoria (nested)
            categorias.append(lista_suporte)
            #procura por uma classe em questão
            empresa = browser.find_element(By.XPATH, '//span[@class="topcard__flavor"]')
            empresas.append(empresa.text)
            #procura por uma classe em questão
            local = browser.find_element(By.XPATH, '//span[@class="topcard__flavor topcard__flavor--bullet"]')
            locais.append(local.text)
            #tempo para que o linkedin não bloqueie nosso acesso.
            time.sleep(3)
            #para fins de organização, o link precisa ter mesmo index que as outras observações
            links.append(link)
        except:
            #as vezes o linkedin pede por usuário.
            contador_01 += 1
            time.sleep(3)
    except:
        contador_02 += 1
print('linkedin pediu por cadastro {} vezes'.format(contador_01))
print('falhou ao conectar {} vezes'.format(contador_02))

linkedin pediu por cadastro 0 vezes
falhou ao conectar 0 vezes


Verificação

In [10]:
print(len(cargos))
print(len(descricoes))
print(len(categorias))
print(len(empresas))
print(len(links))
print(len(locais))

812
812
812
812
812
812


In [12]:
#criação de um dataframe para melhor manipular os dados.
df = pd.DataFrame(list(zip(links, empresas, cargos, categorias, locais, descricoes)), columns=['link','empresas','cargos','categorias','locais', 'descricoes'])

In [13]:
df.shape

(812, 6)

In [14]:
df.to_csv('linkedin_table.csv', index = False)

In [15]:
df.head()

,link,empresas,cargos,categorias,locais,descricoes
0,https://br.linkedin.com/jobs/view/cientista-de...,Neogrid,Cientista de Dados Júnior,"[Assistente, Tempo integral, Engenharia e Tecn...","Agudos, SP",Nós da Neogrid estamos em busca de pessoas que...
1,https://br.linkedin.com/jobs/view/analista-de-...,Riachuelo,Analista de Dados Júnior,"[Não aplicável, Tempo integral, Tecnologia da ...",Brasil,Aqui na Riachuelo a gente se conecta com @vc! ...
2,https://br.linkedin.com/jobs/view/analista-de-...,ProjecTI,Analista de Dados Júnior,[Contrato],"São Paulo, SP","Por favor, ler com atenção a descrição.\n\nVag..."
3,https://br.linkedin.com/jobs/view/analista-de-...,Grupo Junco - Industria e Comercio,Analista de dados,[Tempo integral],"Uberlândia, MG","Buscamos candidato com perfil analítico, boa c..."
4,https://br.linkedin.com/jobs/view/cientista-de...,EY,Cientista de Dados Júnior 1,"[Assistente, Tempo integral, Engenharia e Tecn...","Porto Alegre, RS","Na EY, você terá a chance de construir uma car..."
